In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

$X= ``Número\ de\ gols\ do\ time\ de\ casa" \sim Poisson(\lambda_x)\\$
$Y= ``Número\ de\ gols\ do\ time\ visitante" \sim Poisson(\lambda_x)$

$$P(X=x)=\frac{\lambda^{x}_{x}e^{-\lambda_x}}{x!}\qquad P(Y=y)=\frac{\lambda^{y}_{y}e^{-\lambda_y}}{y!}\\$$

$$P(X=x, Y=y)=P(X=x)P(Y=y)=\frac{\lambda_{x}^{x}\lambda_{y}^{y}e^{-(\lambda_x+\lambda_y)}}{x!y!}$$

In [180]:
tabela = pd.read_excel('dados/tabela/tabela-rodada_26.xlsx')
tabela_casa = pd.read_excel('dados/tabela/tabela_casa-rodada_27.xlsx')
historico_casa = pd.read_excel('dados/historico/historicocasa-rodada_26.xlsx')
historico = pd.read_excel('dados/historico/historico-rodada_26.xlsx')
rodadas = pd.read_excel('dados/previsoes/todas_rodadas.xlsx')
rodadas_futuras = pd.read_excel('dados/previsoes/rodadas_futuras-rodada-27.xlsx')

rodada_atual = tabela['MP'].max()

In [181]:
def gerar_lambda(par_x, par_y, peso_x, peso_y, fator_casa, rodada):
  df = rodadas_futuras[rodadas_futuras['Wk'] == rodada]
  dem_x = (sum(peso_x[0])+sum(peso_x[1]))
  dem_y = (sum(peso_y[0])+sum(peso_y[1]))

  for time_casa, time_fora in zip(df['Home'], df['Away']):
    # Time alvo == Casa
    att_x = sum(tabela.loc[tabela['Squad'] == time_casa, par_x[0][i]]*peso_x[0][i] for i in range(len(par_x[0])))/dem_x
    def_y = sum(tabela.loc[tabela['Squad'] == time_casa, par_x[1][i]]*peso_x[0][i] for i in range(len(par_x[1])))/dem_x
    casa_x = ((1/int(tabela_casa.loc[tabela_casa['Squad'] == time_casa, fator_casa[0]]))*0.5)+1
    lambda_x = att_x*def_y*casa_x
    # Time alvo == Casa
    att_y = sum(tabela.loc[tabela['Squad'] == time_casa, par_y[0][i]]*peso_y[0][i] for i in range(len(par_y[0])))/dem_y
    def_x = sum(tabela.loc[tabela['Squad'] == time_casa, par_y[1][i]]*peso_y[0][i] for i in range(len(par_y[1])))/dem_y
    lambda_y = att_y*def_x
    
    df.loc[df['Home'] == time_casa, 'lambda_x'] = float(lambda_x)
    df.loc[df['Away'] == time_fora, 'lambda_y'] = float(lambda_y)

  return df

In [182]:
def prob_vitoria(rodada_df, rodada, lambdas):
  rodada_df = rodada_df.loc[rodada_df['Wk'] == rodada]
  prob_gol = {}
  times_casa = []
  times_fora = []
  
  # Times casa
  for time in rodada_df['Home']:
    prob_gol[time] = []
    times_casa.append(time)
    for k in range(0, 11):
      prob_gol[time].append(stats.poisson.pmf(k, lambdas.loc[lambdas['Home'] == time, 'lambda_x']))
  
  # Times Fora
  for time in rodada_df['Away']:
    prob_gol[time] = []
    times_fora.append(time)
    for k in range(0, 11):
      prob_gol[time].append(stats.poisson.pmf(k, lambdas.loc[lambdas['Away'] == time, 'lambda_y']))

  casa_vitoria = {}
  fora_vitoria = {}

  for time_casa, time_fora in zip(times_casa, times_fora): 
    casa_vitoria[time_casa] = []
    fora_vitoria[time_fora] = []
    for i in range(1, 11):
      casa_vitoria[time_casa].append(prob_gol[time_casa][i] * sum(prob_gol[time_fora][:i]))
      fora_vitoria[time_fora].append(prob_gol[time_fora][i] * sum(prob_gol[time_casa][:i]))
    casa_vitoria[time_casa] = sum(casa_vitoria[time_casa])
    fora_vitoria[time_fora] = sum(fora_vitoria[time_fora])
    rodada_df.loc[rodada_df['Home'] == time_casa, 'vitoria_casa'] = casa_vitoria[time_casa]
    rodada_df.loc[rodada_df['Away'] == time_fora, 'vitoria_fora'] = fora_vitoria[time_fora]
    rodada_df.loc[rodada_df['Away'] == time_fora, 'empate'] = 1- (rodada_df.loc[rodada_df['Home'] == time_casa, 'vitoria_casa'] + rodada_df.loc[rodada_df['Away'] == time_fora, 'vitoria_fora'])
    

  return rodada_df, prob_gol

In [183]:
def prob_gols(rodadas_prob, prob_gol):
  rodadas_probgols = pd.DataFrame.from_dict(prob_gol, orient='index').astype(float)

  df_casa = pd.DataFrame()
  for time in rodadas_prob['Home']:
    linha_casa = pd.DataFrame(rodadas_probgols.loc[time,]).T
    linha_casa['Home'] = time
    df_casa = pd.concat([df_casa, linha_casa])

  df_fora = pd.DataFrame()
  for time in rodadas_prob['Away']:
    linha_fora = pd.DataFrame(rodadas_probgols.loc[time,]).T
    linha_fora['Away'] = time
    df_fora = pd.concat([df_fora, linha_fora])


  df_qsefinal = pd.merge(rodadas_prob, df_casa, on='Home')
  df_qsefinal = pd.merge(df_qsefinal, df_fora, on='Away')
  df_final = pd.merge(rodadas_futuras, df_qsefinal, on= ['Wk', 'Day', 'Date', 'Home', 'xGCasa', 'gols_casa', 'gols_fora', 'xGFora', 'Away'], how='left')


  return df_final, df_qsefinal

In [184]:
par_x = [['xG_total', 'xG_casa', 'GF_total', 'GF_casa', 'xG_5total', 'xG_5casa', 'GF_5total', 'GF_5casa'], #Ataque casa
        ['GA_total', 'GA_fora', 'xG_total', 'xG_fora', 'xG_5total', 'xG_5fora', 'GA_5total', 'GA_5fora']]  #Defesa fora
pes_x = [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]


par_y = [['xG_total', 'xG_fora', 'GF_total', 'GF_fora', 'xG_5total', 'xG_5fora', 'GF_5total', 'GF_5fora'], #Ataque fora
        ['GA_total', 'GA_casa', 'xG_total', 'xG_casa', 'xG_5total', 'xG_5casa', 'GA_5total', 'GA_5casa']]  #Defesa casa
pes_y = [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]

fator_casa = ['Rk_casa']

In [185]:
lambdas = gerar_lambda(par_x, par_y, pes_x, pes_y, fator_casa, rodada_atual+1)

In [186]:
rodadas_prob, prob_gol = prob_vitoria(rodadas_futuras, rodada_atual+1, lambdas)

In [187]:
lambdas

,Wk,Day,Date,Home,xGCasa,gols_casa,gols_fora,xGFora,Away,lambda_x,lambda_y
0,27,Wed,2023-10-18,Gremio,NaN,NaN,NaN,NaN,Ath Paranaense,0.959737,0.609630
1,27,Wed,2023-10-18,Coritiba,NaN,NaN,NaN,NaN,Cuiaba,0.948397,0.785072
2,27,Wed,2023-10-18,America (MG),NaN,NaN,NaN,NaN,Botafogo (RJ),0.871598,0.819660
3,27,Thu,2023-10-19,Cruzeiro,NaN,NaN,NaN,NaN,Flamengo,0.397524,0.429629
4,27,Thu,2023-10-19,Palmeiras,NaN,NaN,NaN,NaN,Atletico Mineiro,0.741847,0.661391
5,27,Thu,2023-10-19,Santos,NaN,NaN,NaN,NaN,Bragantino,0.895168,0.781953
6,27,Wed,2023-10-18,Bahia,NaN,NaN,NaN,NaN,Internacional,1.074140,1.064393
7,27,Wed,2023-10-18,Vasco da Gama,NaN,NaN,NaN,NaN,Fortaleza,1.086832,0.813839
8,27,Wed,2023-10-18,Goias,NaN,NaN,NaN,NaN,Sao Paulo,0.536785,0.497776
9,27,Thu,2023-10-19,Fluminense,NaN,NaN,NaN,NaN,Corinthians,1.064307,0.678068


In [188]:
rodadas_futuras_prob, rodada_futura_prob = prob_gols(rodadas_prob, prob_gol)

In [189]:
rodadas_prob

,Wk,Day,Date,Home,xGCasa,gols_casa,gols_fora,xGFora,Away,vitoria_casa,vitoria_fora,empate
0,27,Wed,2023-10-18,Gremio,NaN,NaN,NaN,NaN,Ath Paranaense,0.425758,0.225247,0.348995
1,27,Wed,2023-10-18,Coritiba,NaN,NaN,NaN,NaN,Cuiaba,0.377834,0.287346,0.334820
2,27,Wed,2023-10-18,America (MG),NaN,NaN,NaN,NaN,Botafogo (RJ),0.343839,0.314749,0.341412
3,27,Thu,2023-10-19,Cruzeiro,NaN,NaN,NaN,NaN,Flamengo,0.230956,0.253817,0.515227
4,27,Thu,2023-10-19,Palmeiras,NaN,NaN,NaN,NaN,Atletico Mineiro,0.333158,0.284816,0.382027
5,27,Thu,2023-10-19,Santos,NaN,NaN,NaN,NaN,Bragantino,0.360543,0.296956,0.342501
6,27,Wed,2023-10-18,Bahia,NaN,NaN,NaN,NaN,Internacional,0.354331,0.349366,0.296303
7,27,Wed,2023-10-18,Vasco da Gama,NaN,NaN,NaN,NaN,Fortaleza,0.415749,0.270305,0.313946
8,27,Wed,2023-10-18,Goias,NaN,NaN,NaN,NaN,Sao Paulo,0.284563,0.258563,0.456875
9,27,Thu,2023-10-19,Fluminense,NaN,NaN,NaN,NaN,Corinthians,0.443005,0.230811,0.326184


In [190]:
rodadas_futuras_prob.to_excel('dados/previsoes/todas_previsoes.xlsx')
rodada_futura_prob.to_excel(f'dados/previsoes/previsao_rodada-{rodada_atual+1}.xlsx')